### This notebook is inspired by https://github.com/EOPF-Sample-Service/eopf-sample-webinar2/

In [ ]:
import datetime
import os

import numpy as np
import xarray as xr

In [ ]:
from dask.distributed import Client

client = Client(processes=False)
client

In [ ]:
import jp2io.zarr.codec  # register the codec

In [ ]:
cdse_storage_options = dict(
    remote_protocol="s3",
    remote_options=dict(
        key=os.environ["CDSE_ACCESS_KEY_ID"],
        secret=os.environ["CDSE_SECRET_ACCESS_KEY"],
        endpoint_url="https://eodata.dataspace.copernicus.eu",
    ),
)

In [ ]:
ds = xr.open_dataset("./L2A-32UPC.kerchunk", engine="kerchunk", backend_kwargs={"storage_options": cdse_storage_options}, chunks="auto")
ds = ds.astype(np.float32)
ds

In [ ]:
ndvi = (ds.B08 - ds.B04) / (ds.B08 + ds.B04)
ndvi

In [ ]:
img = ndvi.sel(time=datetime.datetime(2018, 7, 11), method="nearest")
img[:300, :300].plot()

In [ ]:
x_slice = slice(1000, 1500)
y_slice = slice(5000, 5500)
yearly_da = (
    ndvi.sel(x=x_slice, y=y_slice)
    [::10] # one image every 50 days
    .groupby("time.year")
    .median(dim="time", skipna=True)
)
yearly_da

In [ ]:
yearly_da.sel(year=2020).plot.imshow(vmin=-0.1, vmax=0.5)